In [ ]:
import toml
from sqlalchemy import create_engine
import pandas as pd
from konlpy.tag import Okt
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [2]:
# connect db
config = toml.load("../config/crawler_config.toml")
db_url = 'mysql://' + config['db']['user'] + ':' + config['db']['password'] + '@' + config['db']['host'] \
                            + '/' + config['db']['db'] + '?charset=utf8'
engine = create_engine(db_url, encoding='utf-8')
conn = engine.connect()

In [3]:
comments_df = pd.read_sql_table('comments', con=conn)

In [4]:
comments_df.loc[comments_df.time == '옵션 열기', 'time'] = None
comments_df.loc[comments_df.time == '답글0', 'time'] = None

In [5]:
# str to Datetime
comments_df.time = pd.to_datetime(comments_df.time, format="%Y.%m.%d. %H:%M")

In [6]:
# make phrases fuction
def make_phrases(content: str) -> str:
    okt = Okt()
    try:
        return okt.phrases(okt.normalize(content))
    except Exception:
        if content is not None:
            print(content)
        return None

In [7]:
# make content phrases token
comments_df['content_phrases'] = comments_df.content.progress_apply(lambda x: make_phrases(x))

In [8]:
comments_df.to_pickle("./comment.pkl")